<a href="https://colab.research.google.com/github/Dr-Amar/Sphalerite-Gunga-Pb-Zn-DeepLearning/blob/main/Amar_K_Fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
import keras
from keras.utils import to_categorical
from google.colab import files

In [ ]:
df = pd.read_excel("/content/Sphalerite whole.xlsx")
df = df.sort_values(by="Deposit")
df.head()

,Number,Deposit,Sample,Deposit Type,Mn,Fe,Co,Cu,Ga,Ge,Ag,Cd,In,Sn,Sb,Pb
798,799,Angouran,AN14-7-9(S1),MVT,33.27,81579.0,276.2,1131.00,202.900,237.5,381.60,5509.0,20.46,73.950,399.2,561.4
821,822,Angouran,AN14-8-4(S2),MVT,2.532,14348.0,224.0,113.30,2.949,381.9,76.29,5873.0,0.027,0.266,5.295,295.0
823,824,Angouran,AN14-8-4(S2),MVT,2.136,11365.0,172.4,76.68,9.123,254.4,62.15,5396.0,0.023,0.716,7.101,413.3
824,825,Angouran,AN14-8-4(S2),MVT,3.528,14572.0,199.0,53.27,1.895,282.5,67.95,5233.0,0.072,0.208,7.068,477.5
825,826,Angouran,AN14-8-4(S2),MVT,3.621,13680.0,219.2,151.60,2.952,464.7,77.34,4930.0,0.002,0.434,7.697,589.5


In [ ]:
df.drop(["Number","Sample"],inplace=True,axis=1)


In [ ]:
df["Mn"] = df["Mn"].replace("<","",regex=True)
df["In"] = df["In"].replace("<","",regex=True)
df["Sb"] = df["Sb"].replace("<","",regex=True)
df["Mn"]=df["Mn"].astype("float64")
df["In"]=df["In"].astype("float64")
df["Sb"]=df["Sb"].astype("float64")

# ***Encoding and sorting data based on Deposit***

In [ ]:
encoder = LabelEncoder()
df["Deposit"] = encoder.fit_transform(df["Deposit"])

In [ ]:
df["Deposit Type"] = encoder.fit_transform(df["Deposit Type"])
#df["Deposit Type"] = to_categorical(df["Deposit Type"],num_classes=5)
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(df["Deposit Type"]),
                                     y=df["Deposit Type"])
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
pd.set_option('display.max_rows', None)

# Print the entire column
print(df['Deposit'])

# Reset display options to default (optional)
pd.reset_option('display.max_rows')

798      0
821      0
823      0
824      0
825      0
826      0
827      0
828      0
820      0
829      0
831      0
832      0
833      0
834      0
835      0
836      0
837      0
830      0
819      0
822      0
817      0
800      0
801      0
802      0
803      0
804      0
818      0
806      0
807      0
808      0
805      0
810      0
811      0
812      0
813      0
814      0
815      0
816      0
809      0
799      0
2454     1
2448     1
2449     1
2450     1
2451     1
2452     1
2453     1
0        2
1        2
2        2
3        2
4        2
5        2
119      3
116      3
117      3
118      3
120      3
121      3
124      3
123      3
115      3
125      3
126      3
127      3
122      3
98       3
101      3
100      3
128      3
102      3
103      3
104      3
105      3
106      3
107      3
108      3
109      3
110      3
111      3
112      3
99       3
129      3
132      3
131      3
130      3
138      3
137      3
136      3
114      3
134      3

In [ ]:
unique_categories = df['Deposit'].nunique()
print(unique_categories)

96


In [ ]:
df.head(20)

,Deposit,Deposit Type,Mn,Fe,Co,Cu,Ga,Ge,Ag,Cd,In,Sn,Sb,Pb
798,0,1,33.270,81579.0,276.2,1131.000,202.900,237.50,381.600,5509.0,20.460,73.950,399.200,561.40
821,0,1,2.532,14348.0,224.0,113.300,2.949,381.90,76.290,5873.0,0.027,0.266,5.295,295.00
823,0,1,2.136,11365.0,172.4,76.680,9.123,254.40,62.150,5396.0,0.023,0.716,7.101,413.30
824,0,1,3.528,14572.0,199.0,53.270,1.895,282.50,67.950,5233.0,0.072,0.208,7.068,477.50
825,0,1,3.621,13680.0,219.2,151.600,2.952,464.70,77.340,4930.0,0.002,0.434,7.697,589.50
826,0,1,8.924,30369.0,258.2,24.270,4.384,103.60,356.600,8522.0,0.152,1.524,294.200,268.10
827,0,1,2.287,11439.0,236.0,10.500,1.217,55.79,51.020,4559.0,0.016,0.729,42.880,33.81
828,0,1,4.182,25823.0,551.7,9.391,1.240,107.40,33.910,551.6,0.012,0.964,10.090,15.56
820,0,1,4.581,12842.0,187.8,23.210,1.446,36.21,24.620,3888.0,0.035,0.650,26.220,35.94
829,0,1,2.364,17104.0,305.6,15.530,1.648,95.40,40.740,1700.0,0.008,0.873,12.500,30.65


# ***Function for spliting data into x and y***

In [ ]:
itr = 0
def feature_split(df_train, df_test):
    # Drop the 'Deposit' column from both train and test sets
    df_train.drop("Deposit", axis=1, inplace=True)
    df_test.drop("Deposit", axis=1, inplace=True)

    # Extract features and labels for the train set
    x_train = df_train.drop("Deposit Type", axis=1)
    y_train = df_train["Deposit Type"]

    # Extract features and labels for the test set
    x_test = df_test.drop("Deposit Type", axis=1)
    y_test = df_test["Deposit Type"]

    # Encode the train set labels (y_train)
   # #
   # label_encoder_train = LabelEncoder()
    #y_encoded_train = label_encoder_train.fit_transform(y_train)
    # Convert the numerical labels to one-hot encoding for the train set
    #y_one_hot_train = to_categorical(y_encoded_train)
    y_one_hot_train = to_categorical(y_train)

    # Encode the test set labels (y_test)
    #label_encoder_test = LabelEncoder()
    #y_encoded_test = label_encoder_test.fit_transform(y_test)
    # Convert the numerical labels to one-hot encoding for the test set
    #y_one_hot_test = to_categorical(y_encoded_test, num_classes=len(label_encoder_train.classes_))
    y_one_hot_test = to_categorical(y_test, num_classes=5)

    # Scale the features for the train set
    scaler = MinMaxScaler()
    x_scaled_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)

    # Get the min and max values
    min_values = scaler.data_min_
    max_values = scaler.data_max_

# Save the min and max values to a file
    np.savez('scaler_params.npz', min=min_values, max=max_values)
    files.download('scaler_params.npz')
    return x_scaled_train,y_one_hot_train, x_test, y_one_hot_test



# ***Function for selecting data for testing and training***

In [ ]:
def train_test_split(df, category):
  df_copy = df.copy()
  df_test = df_copy[df_copy["Deposit"] == category]
  df_train = df_copy[df_copy["Deposit"] != category]
  x_train,y_train,x_test,y_test = feature_split(df_train,df_test)
 # print(f" Shape of data is : {y_train[1]} \t {y_test[1]}")
  return x_train, y_train, x_test, y_test

# ***Function for evaluating the model***

In [ ]:
from sklearn.metrics import precision_score, recall_score

def evaluate_model(model, x_test, y_test):
    y_pred_prob = model.predict(x_test)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_test_categorical = np.argmax(y_test, axis=1)  # Convert one-hot encoded back to categorical labels
    score = model.evaluate(x_test, y_test)
    precision = precision_score(y_test_categorical, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test_categorical, y_pred, average='weighted', zero_division=0)
    return precision, recall, score


# ***Model***

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

In [ ]:
input = 12
hidden1 = 400
hidden2 = 200
hidden3 = 150
hidden4 = 32
hidden5 = 16
classes = 5
epochs = 500


In [ ]:
from keras.optimizers import Adam
from keras.layers import Dropout, BatchNormalization
from keras.metrics import Precision, Recall

# Define the optimizer
optimizer_sc = Adam(learning_rate=0.0001)


model_sc = Sequential()
model_sc.add(Dense(hidden1,activation = "tanh",input_dim = input))
model_sc.add(BatchNormalization())
model_sc.add(Dropout(0.1))  # Add dropout after each dense layer

model_sc.add(Dense(hidden2, activation = "tanh"))
model_sc.add(BatchNormalization())
model_sc.add(Dropout(0.1))  # Add dropout after each dense layer

model_sc.add(Dense(hidden3, activation = "tanh"))
model_sc.add(BatchNormalization())
model_sc.add(Dropout(0.1))  # Add dropout after each dense layer

model_sc.add(Dense(hidden4, activation = "tanh"))
model_sc.add(BatchNormalization())
model_sc.add(Dropout(0.1))  # Add dropout after each dense layer

model_sc.add(Dense(hidden5, activation = "tanh"))
model_sc.add(BatchNormalization())
model_sc.add(Dropout(0.1))  # Add dropout after each dense layer

model_sc.add(Dense(classes, activation = "softmax"))
early_stopping = EarlyStopping(monitor='accuracy', patience=20, restore_best_weights=True)
#optimizer_sc = keras.optimizers.Adam(learning_rate=0.001)

model_sc.compile(loss="categorical_crossentropy",optimizer = optimizer_sc,metrics = [Precision(),Recall(),"accuracy"])
model_sc.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 400)                 │           5,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 400)                 │           1,600 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 400)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 200)                 │          80,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 200)                 │             800 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 150)                 │          30,150 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 150)                 │             600 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 150)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 32)                  │           4,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 16)                  │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 124,187 (485.11 KB)

 Trainable params: 122,591 (478.87 KB)

 Non-trainable params: 1,596 (6.23 KB)

# ***Model Training and evaluation for each category***

In [ ]:
from logging import logProcesses
accuracy = []
precisionList = []
recallList = []
lossList = []
for n in range(unique_categories):
 # model_sc.reset_states()
  x_train, y_train, x_test, y_test = train_test_split(df,n)
  print(f"Iteration number :{n}")
  model_sc.fit(x_train,y_train, epochs = epochs,callbacks=[early_stopping], verbose = 1)
  precision, recall, score = evaluate_model(model_sc,x_test,y_test)
  accuracy.append(score)
  precisionList.append(precision)
  recallList.append(recall)
  #lossList.append(loss)


<ipython-input-16-6ff18684b05e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop("Deposit", axis=1, inplace=True)
<ipython-input-16-6ff18684b05e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop("Deposit", axis=1, inplace=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Iteration number :0
Epoch 1/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.3666 - loss: 1.5475 - precision_1: 0.4466 - recall_1: 0.1865
Epoch 2/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6209 - loss: 0.9961 - precision_1: 0.7404 - recall_1: 0.4739
Epoch 3/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6656 - loss: 0.9137 - precision_1: 0.7807 - recall_1: 0.5413
Epoch 4/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6764 - loss: 0.8794 - precision_1: 0.7703 - recall_1: 0.5613
Epoch 5/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6890 - loss: 0.8486 - precision_1: 0.7789 - recall_1: 0.5874
Epoch 6/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6985 - loss: 0.8210 - precision_1: 0.7823 - recall_1: 0.6082
Epoch 7/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7219 - loss: 0.7715 - precision_1: 0.7970 - recall_1: 0.6359
Epoch 8/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7312 - loss: 0.7

<ipython-input-16-6ff18684b05e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop("Deposit", axis=1, inplace=True)
<ipython-input-16-6ff18684b05e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop("Deposit", axis=1, inplace=True)


...

Iteration number :94
Epoch 1/500
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9287 - loss: 0.1981 - precision_1: 0.9327 - recall_1: 0.9240
Epoch 2/500
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9299 - loss: 0.2094 - precision_1: 0.9341 - recall_1: 0.9240
Epoch 3/500
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9228 - loss: 0.2013 - precision_1: 0.9288 - recall_1: 0.9152
Epoch 4/500
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9195 - loss: 0.2028 - precision_1: 0.9240 - recall_1: 0.9146
Epoch 5/500
119/119 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9276 - loss: 0.1975 - precision_1: 0.9330 - recall_1: 0.9251
Epoch 6/500
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9351 - loss: 0.1826 - precision_1: 0.9388 - recall_1: 0.9308
Epoch 7/500
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9331 - loss: 0.1835 - precision_1: 0.9368 - recall_1: 0.9282
Epoch 8/500
119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9294 - loss:

<ipython-input-16-6ff18684b05e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop("Deposit", axis=1, inplace=True)
<ipython-input-16-6ff18684b05e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop("Deposit", axis=1, inplace=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Iteration number :95
Epoch 1/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9239 - loss: 0.2170 - precision_1: 0.9267 - recall_1: 0.9188
Epoch 2/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9134 - loss: 0.2366 - precision_1: 0.9179 - recall_1: 0.9091
Epoch 3/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9319 - loss: 0.1947 - precision_1: 0.9364 - recall_1: 0.9276
Epoch 4/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9376 - loss: 0.1823 - precision_1: 0.9422 - recall_1: 0.9342
Epoch 5/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9135 - loss: 0.2175 - precision_1: 0.9179 - recall_1: 0.9125
Epoch 6/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9244 - loss: 0.1971 - precision_1: 0.9275 - recall_1: 0.9185
Epoch 7/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9320 - loss: 0.1870 - precision_1: 0.9346 - recall_1: 0.9270
Epoch 8/500
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9258 - loss: 0.1

In [ ]:

import numpy as np
from sklearn.preprocessing import LabelEncoder

# If y_train is one-hot encoded, convert it to class indices
y_train_class_indices = np.argmax(y_train, axis=1)

# Initialize LabelEncoder and fit to the original class indices
label_encoder = LabelEncoder()
label_encoder.fit(y_train_class_indices)

# Get the class names and their encoded labels
class_names = label_encoder.classes_

# Create the encoded labels list
encoded_labels = list(zip(class_names, range(len(class_names))))

# Print the encoded labels
print(encoded_labels)

# Print each encoded value and its corresponding class name
for i, class_name in enumerate(class_names):
    print(f"Encoded Value: {i}, Class Name: {class_name}")



[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
Encoded Value: 0, Class Name: 0
Encoded Value: 1, Class Name: 1
Encoded Value: 2, Class Name: 2
Encoded Value: 3, Class Name: 3
Encoded Value: 4, Class Name: 4


In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# If y_train is one-hot encoded, convert it to class indices
y_train_class_indices = np.argmax(y_train, axis=1)

# Initialize LabelEncoder and fit to the original class indices
label_encoder = LabelEncoder()
label_encoder.fit(y_train_class_indices)

# Get the class names and their encoded labels
class_names = label_encoder.classes_

# Print the encoded labels directly by zipping the class names with their indices
encoded_labels = list(zip(range(len(class_names)), class_names))

# Print the encoded labels
print(encoded_labels)

# Print each encoded value and its corresponding class name
for i, class_name in enumerate(class_names):
    print(f"Encoded Value: {i}, Class Name: {class_name}")


[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
Encoded Value: 0, Class Name: 0
Encoded Value: 1, Class Name: 1
Encoded Value: 2, Class Name: 2
Encoded Value: 3, Class Name: 3
Encoded Value: 4, Class Name: 4


In [ ]:

print(np.mean(precisionList))
print(np.mean(recallList))
print(np.mean(accuracy))



0.8001005448884276
0.48327966513160286
0.8406216222144091


# ***DOwnloading Accuracy***


In [ ]:
accuracy_array = np.array(accuracy)

# Save accuracy array to a file
np.savetxt('accuracy_array.csv', accuracy_array, delimiter=',')

# Download the accuracy array file
#files.download('accuracy_array.csv')

# ***Downloading Model***

In [ ]:
from google.colab import files
model_sc.save("geological_geposit_predictor2.h5")
files.download("geological_geposit_predictor2.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for itr,c in enumerate(precisionList):
  print(f"{precisionList[itr]} \t {recallList[itr]} \t {accuracy[itr]}")

1.0 	 1.0 	 [0.0031593882013112307, 1.0, 1.0, 1.0]
0.0 	 0.0 	 [5.5933942794799805, 0.0, 0.0, 0.0]
1.0 	 0.6666666666666666 	 [0.6344233155250549, 0.6666666865348816, 0.6666666865348816, 0.6666666865348816]
1.0 	 0.5853658536585366 	 [1.9627306461334229, 0.5853658318519592, 0.5853658318519592, 0.5853658318519592]
1.0 	 0.15 	 [5.954127311706543, 0.1666666716337204, 0.15000000596046448, 0.15000000596046448]
0.0 	 0.0 	 [7.479183673858643, 0.0, 0.0, 0.0]
1.0 	 0.795774647887324 	 [0.4977115988731384, 0.8257575631141663, 0.7676056623458862, 0.7957746386528015]
0.0 	 0.0 	 [4.729509353637695, 0.0, 0.0, 0.0]
0.0 	 0.0 	 [3.049967050552368, 0.0, 0.0, 0.0]
1.0 	 0.9880952380952381 	 [0.04589860141277313, 0.988095223903656, 0.988095223903656, 0.988095223903656]
1.0 	 0.8977272727272727 	 [0.4009462594985962, 0.9080459475517273, 0.8977272510528564, 0.8977272510528564]
1.0 	 0.3076923076923077 	 [3.303269624710083, 0.31111112236976624, 0.26923078298568726, 0.3076923191547394]
0.0 	 0.0 	 [7.5342

In [ ]:
def k_fold_trainig(x_train,x_text,y_train,y_test):
  model_sc.fit(x_train,y_train,callbacks=[early_stopping], epochs = epochs, verbose = 1)

# ***K-FoldTraditional***

In [ ]:
# Instantiate KFold
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Perform k-fold cross-validation
fold_scores = []
for fold, (train_index, test_index) in enumerate(skf.split(x, x["Deposit"])):

    X_train, X_test = x.iloc[train_index], x.iloc[test_index]  # Access data using .iloc[]
    y_train, y_test = y_one_hot[train_index], y_one_hot[test_index]

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Evaluate the model
    _, score = model.evaluate(X_test, y_test, verbose=0)
    fold_scores.append(score)
    print(f'Score for fold {fold+1}: {score}')

# Calculate average score
average_score = np.mean(fold_scores)
print(f'Average score across {k_folds} folds: {average_score}')
